In [85]:
#program to compare matrix multiplication using sequential and parallel methods

using Pkg
Pkg.add("CUDAnative")
Pkg.add("CuArrays")
Pkg.add("CUDAdrv")
Pkg.add("BenchmarkTools")
Pkg.add("Test")

using CUDAnative,CuArrays,CUDAdrv,BenchmarkTools,Test

n=parse(Int64,readline())
a=rand(Float32,n,n)
b=rand(Float32,n,n)
c1=zeros(Float32,n,n)
c2=zeros(Float32,n,n)

#function to calculate matrix multiplication using sequential code

function mult_seq!(a,b,c,n)
for i=1:n
        for j=1:n
            x=0;
            for k=1:n
                  x=x+a[(k-1)*n+i]*b[(j-1)*n+k]
            end
            c[i+(j-1)*n]=x;
        end
    end
end

@btime mult_seq!(a,b,c1,n)

#function to calculate matrix multiplication using parallel code

function mult_par!(a,b,c,n)
tidx=threadIdx().x
bidx=blockIdx().x
sum=0
for i=1:n
sum=sum+a[i+(n-1)*(i-1)+(bidx-1)]*b[i+n*(tidx-1)]
end
c[bidx+(tidx-1)*n]=sum
return
end

d_a=CuArray(a)
d_b=CuArray(b)
d_c=CuArray(c2)
@btime @cuda blocks=n threads=n mult_par!(d_a,d_b,d_c,n)

h_c2=Array(d_c)

@test isapprox(c1,h_c2)

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
stdin> 1000
  1.558 s (0 allocations: 0 bytes)
  13.112 μs (66 allocations: 2.33 KiB)


Test Passed